# Solcore: QW Absorption

The objective of this tutorial is to compute absorption coefficient of InGaAs QW of fixed thickness for five different material compositions. The following code produces Fig. 7 of Ref. [1].

[1] Alonso-Alvarez, D., Wilson, T., Pearce, P. et al. J Comput Electron (2018) 17: 1099. https://doi.org/10.1007/s10825-018-1171-3

*Notebook developed at Solcore Boot Camp 2018, Imperial College London*

We start by importing the modules needed in the tutorial

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
# Show the plots in the notebook
# %matplotlib inline
# Set the font size in the plots
matplotlib.rcParams.update({'font.size': 22})
import numpy as np

from solcore import si, material
from solcore.structure import Layer, Structure
import solcore.quantum_mechanics as QM
from solcore.constants import vacuum_permittivity, q

## Definition of materials and structure
The materials for bulk and barrier are GaAs and GaAs$_{1-x}$P$_x$, respectively. We also set the quantum well thickness $L_z$ here and the different layers with which the solar cell structure is constructed.

In [ ]:
# First we create the materials we need
bulk = material("GaAs")(T=293, strained=False)
barrier = material("GaAsP")(T=293, P=0.1, strained=True)

# QW thickness
Lz = si("7.2nm")

# As well as some of the layers
top_layer = Layer(width=si("30nm"), material=bulk)
inter     = Layer(width=Lz, material=bulk)
barrier_layer = Layer(width=si("15nm"), material=barrier)
bottom_layer  = top_layer

## Computational details
We define here the compositions to be explored, the spectral range and the 2D Exciton details. The well width is used for computing the 2D DOS. Theta is the polar angle to compute the matrix elements. hwhm is the full width at half maximum and dimensionality is 0.5 for pure 2D and 1 for 3D. In general, it is considered a fitting parameter.

In [ ]:
comp = np.linspace(0.05, 0.25, 5)
E = np.linspace(1.15, 1.5, 300) * q
alpha_params = {
    "well_width": Lz, 
    "theta": 0,
    "eps": 12.9 * vacuum_permittivity,
    "espace": E,
    "hwhm": si("6meV"),
    "dimensionality": 0.16,
    "line_shape": "Gauss"
}

## Absorption without exciton contribution
Here we run the main loop to compute the absorption. The quantum well material is In$_x$Ga$_{1-x}$As. `Solcore` always computes the absorption of the QW with the exciton correction. To remove its effect on the spectrum, we need to modify the parameters. A very large dielectric function completely screens the Coulomb interaction.

In [ ]:
alpha_params['eps'] = 1.0E20

plt.figure(figsize=(18,10))
for j, i in enumerate(comp[0:2]):
    # We create the QW material at the given composition
    QW = material("InGaAs")(T=293, In=i, strained=True)

    # And the layer
    well_layer = Layer(width=Lz, material=QW)

    # The following lines create the QW structure, with different number of QWs and interlayers
    test_structure = Structure([barrier_layer, inter, well_layer, inter, barrier_layer], substrate=bulk)

    # Finally, the quantum properties are claculated here
    output = QM.schrodinger(test_structure, quasiconfined=0, num_eigenvalues=20, alpha_params=alpha_params,
                            calculate_absorption=True)

    alfa = output[0]['alphaE'](E)
    plt.plot(1240 / (E / q), alfa / 100, label='{}%'.format(int(i * 100)))

plt.xlim(826, 1100)
plt.ylim(0, 23000)
plt.xlabel('Wavelength (nm)')
plt.ylabel('$\\alpha$ cm$^{-1}$')
plt.legend(loc='upper right', frameon=False)
plt.tight_layout()

plt.show()

## Absorption with exciton contribution
To recover the excitonic effects, it is enough to set the dielectric constant to a realistic value.

In [ ]:
alpha_params['eps'] = 12.9 * vacuum_permittivity

plt.figure(figsize=(18,10))
for j, i in enumerate(comp):
    # We create the QW material at the given composition
    QW = material("InGaAs")(T=293, In=i, strained=True)

    # And the layer
    well_layer = Layer(width=Lz, material=QW)

    # The following lines create the QW structure, with different number of QWs and interlayers
    test_structure = Structure([barrier_layer, inter, well_layer, inter, barrier_layer], substrate=bulk)

    # Finally, the quantum properties are claculated here
    output = QM.schrodinger(test_structure, quasiconfined=0, num_eigenvalues=20, alpha_params=alpha_params,
                            calculate_absorption=True)

    alfa = output[0]['alphaE'](E)
    plt.plot(1240 / (E / q), alfa / 100, label='{}%'.format(int(i * 100)))

plt.xlim(826, 1100)
plt.ylim(0, 23000)
plt.xlabel('Wavelength (nm)')
plt.ylabel('$\\alpha$ cm$^{-1}$')
plt.legend(loc='upper right', frameon=False)
plt.tight_layout()

plt.show()